In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lmfit
import copy    # for append with the fit-objects
from lmfit import models
from scipy import constants

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 30 days


# $\frac{1}{r^3}$ Abstands untersuchung

In [2]:
data_r_hoch_3 = pd.DataFrame([[100,6,5,.5],[150,2.7,7,.5],[200,1.4,7,.5]],
                             columns=['Abstand','Delta_U','Abstand_err','Delta_U_err'])

In [3]:
data_r_hoch_3

,Abstand,Delta_U,Abstand_err,Delta_U_err
0,100,6.0,5,0.5
1,150,2.7,7,0.5
2,200,1.4,7,0.5


In [4]:
def r_hoch_3(x,alpha):
    return alpha/x**3

In [5]:
mod_r_hoch_3 = models.Model(r_hoch_3)
pars_r_hoch_3 = mod_r_hoch_3.make_params()
pars_r_hoch_3['alpha'].set(9112500)

In [6]:
out_r_hoch_3 = mod_r_hoch_3.fit(data_r_hoch_3.Delta_U,pars_r_hoch_3,x=data_r_hoch_3.Abstand,weights=1/data_r_hoch_3.Delta_U_err)

In [7]:
print(out_r_hoch_3.fit_report())

[[Model]]
    Model(r_hoch_3)
[[Fit Statistics]]
    # function evals   = 6
    # data points      = 3
    # variables        = 1
    chi-square         = 4.636
    reduced chi-square = 2.318
[[Variables]]
    alpha:   6.3213e+06 +/- 7.25e+05 (11.46%) (init= 9112500)
[[Correlations]] (unreported correlations are <  0.100)



In [8]:
x = np.arange(50,250,0.1)

In [9]:
plt.plot(x,out_r_hoch_3.eval(x=x),label='Fit')
plt.errorbar(data_r_hoch_3['Abstand'],data_r_hoch_3['Delta_U'],data_r_hoch_3['Delta_U_err'],data_r_hoch_3['Abstand_err'],'. ',label='Messdaten')
plt.legend(loc='best', fontsize=34)
plt.xlim([90,250])
plt.ylim([0,10])
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)

plt.show()

# Empfindlichkeit quantitaiv
$ B = \mu_0 \frac{r^2}{2\left(r^2+d^2\right)^\frac{3}{2}} \cdot I $
<br>
$ \Phi = B \cdot A$

In [10]:
r = 3.5/100
abstand = 10.9/100
A = 0.2/1000**2
I = 0.005
err_r = 0.2/100
err_abstand = 0.5/100

In [11]:
def calc_B(r,d,I):
    μ_0 = constants.mu_0
    return I * μ_0 * r**2/(2*(r**2+d**2)**(3/2))

In [12]:
def calc_delta_B(r,d,I,delta_r,delta_d):
    μ_0 = constants.mu_0
    factor = μ_0*I/2
    first_term = ((2*d**2*r-r**3)/(d**2+r**2)**(5/2))*delta_r
    secound_term = ((3*r**2*d)/(d**2+r**2)**(5/2))*delta_d
    return factor*(first_term**2+secound_term**2)**0.5

In [13]:
B = calc_B(r,abstand,I)

In [14]:
delta_B = calc_delta_B(r,abstand,I,err_r,err_abstand)

In [15]:
B

2.5649607039584348e-09

In [16]:
delta_B

4.0732504384092344e-10

In [17]:
Φ = B*A

In [18]:
Φ

5.12992140791687e-16

In [19]:
delta_Φ = A*delta_B

In [20]:
delta_Φ

8.146500876818469e-17

In [21]:
#Größe einens Flussquants in Wb
flussquant = constants.h/(2*constants.e)
print(flussquant)

2.0678337565123478e-15


In [22]:
Φ/flussquant

0.248081906573046

# Kalibrierung

In [23]:
# Daten übertragen
kalib_data = []
columns = ['0 mA','10 mA','20 mA','30 mA','40 mA','50 mA','60 mA','70 mA','80 mA','90 mA']
index = ['1k','err_1k','1.5k','err_1.5k','2k','err_2k','3k','err_3k','4k','err_4k','6k','err_6k','10k','err_10k','20k','err_20k']
current = np.arange(0,100,10)
err_current = np.ones(10)*2
# 1k Widerstandswerte
kalib_data.append([0,208/5,208/5*2.1,208/5*3.3,208/5*4.6,208/5*5.5,208/5*6.4,208/5*7.2,208/5*8.5,208/5*9.8])
kalib_data.append(np.ones(10)*208/20)
# 1.5k Widerstandswerte
kalib_data.append([0,208/5*1.9,208/5*3.4,208/5*4.9,208/5*6.1,208/5*8,208/5*10,208/5*11.2,208/5*13,208/5*14.9])
kalib_data.append(np.ones(10)*208/20)
# 2k Widerstandswerte
kalib_data.append([0,208/5*2.3,208/5*4.1,208/5*6.6,208/5*9,208/5*11.4,208/5*13.7,208/5*15.3,208/5*17.7,208/5*19.8])
kalib_data.append(np.ones(10)*208/20)
# 3k Widerstandswerte
kalib_data.append([0,208/5*3,208/5*6.5,208/5*9.3,208/5*12.5,208/5*15.7,208/5*19,208/5*22.2,208/5*25.1,208/5*28.3])
kalib_data.append(np.ones(10)*208/20)
# 4k Widerstandswerte
kalib_data.append([0,260/5*3.7,260/5*7,260/5*10.1,260/5*13.5,260/5*17,260/5*20.6,260/5*24,260/5*27.3,260/5*31])
kalib_data.append(np.ones(10)*260/20)
# 6k Widerstandswerte
kalib_data.append([0,320/5*4,320/5*8,320/5*12.3,320/5*16,320/5*20,320/5*24,320/5*27.9,320/5*32.1,320/5*36.3])
kalib_data.append(np.ones(10)*320/20)
# 10k Widerstandswerte
kalib_data.append([0,540/5*4,540/5*8.1,540/5*11.9,540/5*16,540/5*20,540/5*24,540/5*27.8,540/5*31.2,540/5*35.1])
kalib_data.append(np.ones(10)*540/20)
# 20k Widerstandswerte
kalib_data.append([0,1100/5*3.7,1100/5*7.5,1100/5*11.1,1100/5*15.1,1100/5*18.9,1100/5*23.7,1100/5*26.5,1100/5*30.1,1100/5*33.9])
kalib_data.append(np.ones(10)*1100/20)

kalib_df = pd.DataFrame([list(i) for i in kalib_data], columns=columns, index=index)

In [24]:
kalib_df.T

,1k,err_1k,1.5k,err_1.5k,2k,err_2k,3k,err_3k,4k,err_4k,6k,err_6k,10k,err_10k,20k,err_20k
0 mA,0.00,10.4,0.00,10.4,0.00,10.4,0.00,10.4,0.0,13,0.0,16,0.0,27,0,55
10 mA,41.60,10.4,79.04,10.4,95.68,10.4,124.80,10.4,192.4,13,256.0,16,432.0,27,814,55
20 mA,87.36,10.4,141.44,10.4,170.56,10.4,270.40,10.4,364.0,13,512.0,16,874.8,27,1650,55
30 mA,137.28,10.4,203.84,10.4,274.56,10.4,386.88,10.4,525.2,13,787.2,16,1285.2,27,2442,55
40 mA,191.36,10.4,253.76,10.4,374.40,10.4,520.00,10.4,702.0,13,1024.0,16,1728.0,27,3322,55
50 mA,228.80,10.4,332.80,10.4,474.24,10.4,653.12,10.4,884.0,13,1280.0,16,2160.0,27,4158,55
60 mA,266.24,10.4,416.00,10.4,569.92,10.4,790.40,10.4,1071.2,13,1536.0,16,2592.0,27,5214,55
70 mA,299.52,10.4,465.92,10.4,636.48,10.4,923.52,10.4,1248.0,13,1785.6,16,3002.4,27,5830,55
80 mA,353.60,10.4,540.80,10.4,736.32,10.4,1044.16,10.4,1419.6,13,2054.4,16,3369.6,27,6622,55
90 mA,407.68,10.4,619.84,10.4,823.68,10.4,1177.28,10.4,1612.0,13,2323.2,16,3790.8,27,7458,55


In [25]:
mod_lin = models.LinearModel()

In [26]:
list_with_fit_objects = []
for i in range(0,16,2):
    temp_para = mod_lin.guess(kalib_df.iloc[i],x=current)
    temp_out_lin = mod_lin.fit(kalib_df.iloc[i],temp_para,x=current,weights=1/kalib_df.iloc[i+1])
    list_with_fit_objects.append(copy.copy(temp_out_lin))
    
    # Ergeniss schön plotten
    plt.errorbar(current,kalib_df.iloc[i],kalib_df.iloc[i+1],err_current,  'b x', label='Messdaten für ' + str(kalib_df.iloc[i].name) + r'$\Omega$')
    plt.plot(np.arange(-10,101,0.1),temp_out_lin.eval(x=np.arange(-10,101,0.1)), 'g-', label='Linearer Fit')
    plt.xlabel('Strom/mA', fontsize=24)
    plt.ylabel('Spannung/mV', fontsize=24)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlim([-10,100])
    plt.legend(loc='best')
    plt.show()

In [27]:
for fit,datensatz in zip(list_with_fit_objects,index[:4:2]):
    print('\t\t\t\tFit für einen Widerstand von ' + datensatz)
    print(fit.fit_report())

				Fit für einen Widerstand von 1k
[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 6
    # data points      = 10
    # variables        = 2
    chi-square         = 4.003
    reduced chi-square = 0.500
[[Variables]]
    slope:       4.44741834 +/- 0.080993 (1.82%) (init= 4.447418)
    intercept:   1.21017121 +/- 4.323862 (357.29%) (init= 1.210182)
[[Correlations]] (unreported correlations are <  0.100)
    C(slope, intercept)          = -0.843 

				Fit für einen Widerstand von 1.5k
[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 6
    # data points      = 10
    # variables        = 2
    chi-square         = 6.968
    reduced chi-square = 0.871
[[Variables]]
    slope:       6.75684822 +/- 0.106861 (1.58%) (init= 6.756848)
    intercept:   1.28583470 +/- 5.704869 (443.67%) (init= 1.285818)
[[Correlations]] (unreported correlations are <  0.100)
    C(slope, intercept)          = -0.843 



In [28]:
for fit,datensatz in zip(list_with_fit_objects,index[4:8:2]):
    print('\t\t\t\tFit für einen Widerstand von ' + datensatz)
    print(fit.fit_report())

				Fit für einen Widerstand von 2k
[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 6
    # data points      = 10
    # variables        = 2
    chi-square         = 4.003
    reduced chi-square = 0.500
[[Variables]]
    slope:       4.44741834 +/- 0.080993 (1.82%) (init= 4.447418)
    intercept:   1.21017121 +/- 4.323862 (357.29%) (init= 1.210182)
[[Correlations]] (unreported correlations are <  0.100)
    C(slope, intercept)          = -0.843 

				Fit für einen Widerstand von 3k
[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 6
    # data points      = 10
    # variables        = 2
    chi-square         = 6.968
    reduced chi-square = 0.871
[[Variables]]
    slope:       6.75684822 +/- 0.106861 (1.58%) (init= 6.756848)
    intercept:   1.28583470 +/- 5.704869 (443.67%) (init= 1.285818)
[[Correlations]] (unreported correlations are <  0.100)
    C(slope, intercept)          = -0.843 



In [29]:
for fit,datensatz in zip(list_with_fit_objects,index[8:12:2]):
    print('\t\t\t\tFit für einen Widerstand von ' + datensatz)
    print(fit.fit_report())

				Fit für einen Widerstand von 4k
[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 6
    # data points      = 10
    # variables        = 2
    chi-square         = 4.003
    reduced chi-square = 0.500
[[Variables]]
    slope:       4.44741834 +/- 0.080993 (1.82%) (init= 4.447418)
    intercept:   1.21017121 +/- 4.323862 (357.29%) (init= 1.210182)
[[Correlations]] (unreported correlations are <  0.100)
    C(slope, intercept)          = -0.843 

				Fit für einen Widerstand von 6k
[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 6
    # data points      = 10
    # variables        = 2
    chi-square         = 6.968
    reduced chi-square = 0.871
[[Variables]]
    slope:       6.75684822 +/- 0.106861 (1.58%) (init= 6.756848)
    intercept:   1.28583470 +/- 5.704869 (443.67%) (init= 1.285818)
[[Correlations]] (unreported correlations are <  0.100)
    C(slope, intercept)          = -0.843 



In [30]:
for fit,datensatz in zip(list_with_fit_objects,index[12:16:2]):
    print('\t\t\t\tFit für einen Widerstand von ' + datensatz)
    print(fit.fit_report())

				Fit für einen Widerstand von 10k
[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 6
    # data points      = 10
    # variables        = 2
    chi-square         = 4.003
    reduced chi-square = 0.500
[[Variables]]
    slope:       4.44741834 +/- 0.080993 (1.82%) (init= 4.447418)
    intercept:   1.21017121 +/- 4.323862 (357.29%) (init= 1.210182)
[[Correlations]] (unreported correlations are <  0.100)
    C(slope, intercept)          = -0.843 

				Fit für einen Widerstand von 20k
[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 6
    # data points      = 10
    # variables        = 2
    chi-square         = 6.968
    reduced chi-square = 0.871
[[Variables]]
    slope:       6.75684822 +/- 0.106861 (1.58%) (init= 6.756848)
    intercept:   1.28583470 +/- 5.704869 (443.67%) (init= 1.285818)
[[Correlations]] (unreported correlations are <  0.100)
    C(slope, intercept)          = -0.843 



In [31]:
widerstand = [1,1.5,2,3,4,6,10,20]
empfindlichkeit = [4.4,6.8,9.2,13.11,17.8,25.7,42.2,83.5]
err_empfindlichkeit = [0.8,0.1,0.1,0.06,0.1,0.1,0.3,0.9]

In [32]:
lin_guess = mod_lin.guess(empfindlichkeit,x=widerstand)
lin_out = mod_lin.fit(empfindlichkeit,lin_guess,x=widerstand,weights=1/np.array(err_empfindlichkeit))

In [33]:
print(lin_out.fit_report())

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 8
    # data points      = 8
    # variables        = 2
    chi-square         = 26.453
    reduced chi-square = 4.409
[[Variables]]
    slope:       4.17623030 +/- 0.045334 (1.09%) (init= 4.144171)
    intercept:   0.69214416 +/- 0.171782 (24.82%) (init= 0.7327361)
[[Correlations]] (unreported correlations are <  0.100)
    C(slope, intercept)          = -0.885 



In [34]:
plt.errorbar(widerstand, empfindlichkeit,yerr=err_empfindlichkeit, fmt='b x', label='Gefittete Empfindlichkeiten')
plt.plot(np.arange(0,21,0.1),lin_out.eval(x=np.arange(0,21,0.1)), 'g-', label='Linearer Fit')
plt.xlabel(r'Widerstand/k$\Omega$', fontsize=24)
plt.ylabel('Empfindlichkeit A/V', fontsize=24)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.xlim([0,21])
plt.legend(loc='best')
plt.show()

## U-B-Diagramm erzeugen

In [35]:
# in \mu Tesla, daher der Faktor 10**6
b_in_abhängigkeit_von_i = calc_B(r,abstand,current/1000)

In [36]:
b_in_abhängigkeit_von_i*10**6

array([ 0.        ,  0.00512992,  0.01025984,  0.01538976,  0.02051969,
        0.02564961,  0.03077953,  0.03590945,  0.04103937,  0.04616929])

In [37]:
delta_b_in_abhängigkeit_von_i = calc_delta_B(r,abstand,current,err_r,err_abstand/1000)

In [65]:
delta_b_in_abhängigkeit_von_i

array([  0.00000000e+00,   5.04079405e-07,   1.00815881e-06,
         1.51223822e-06,   2.01631762e-06,   2.52039703e-06,
         3.02447643e-06,   3.52855584e-06,   4.03263524e-06,
         4.53671465e-06])

In [39]:
plt.errorbar(kalib_df.ix['20k']/1000,b_in_abhängigkeit_von_i*10**6,delta_b_in_abhängigkeit_von_i*10**3, fmt=' x')
plt.show()

In [40]:
def invers_lin(x,a,b):
    return (x-b)/a

In [53]:
mod_invers_lin = models.Model(invers_lin)
pars_invers_lin = mod_invers_lin.make_params()
pars_invers_lin['a'].set(160)
pars_invers_lin['b'].set(0)
out_invers_lin = mod_invers_lin.fit(b_in_abhängigkeit_von_i*10**6,pars_invers_lin,x=kalib_df.ix['20k']/1000)

In [54]:
print(out_invers_lin.fit_report())

[[Model]]
    Model(invers_lin)
[[Fit Statistics]]
    # function evals   = 12
    # data points      = 10
    # variables        = 2
    chi-square         = 0.000
    reduced chi-square = 0.000
[[Variables]]
    a:   162.986680 +/- 1.755798 (1.08%) (init= 160)
    b:  -0.01148986 +/- 0.048078 (418.44%) (init= 0)
[[Correlations]] (unreported correlations are <  0.100)
    C(a, b)                      = -0.843 



In [73]:
plt.plot(kalib_df.ix['20k']/1000,out_invers_lin.eval(), label='Fit')
plt.errorbar(kalib_df.ix['20k']/1000,b_in_abhängigkeit_von_i*10**6,yerr=delta_b_in_abhängigkeit_von_i*10**3,fmt=' x', label='Messdaten')
plt.xlabel(r'B / $\mu$T', fontsize=24)
plt.ylabel('Spannung / V', fontsize=24)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(loc='best')
plt.show()

In [80]:
1/163*4.52

0.02773006134969325

In [85]:
(((4.5*2)/163)**2+(0.1/(163**2))**2)**0.5

0.05521472405466206

In [89]:
def mu(z_6,z_13,d,delta_b):
    factor = 4*constants.pi/constants.mu_0
    upper = (z_13**2+d**2)**(5/2)*(z_6**2+d**2)**(5/2)
    lower = (2*z_13**2-d**2)*(z_6**2+d**2)**(5/2) - (2*z_6**2-d**2)*(z_13**2+d**2)**(5/2)
    return factor*delta_b*(upper/lower)

In [95]:
mu(-4*5,2*4,12,0.0277*10**-6)

-2295.2768194902696